# Polynomial Regression & Overfitting Analysis

## Problem Statement

We want to fit a polynomial curve to noisy data sampled from a sinusoidal function.
The goal is to observe how **model complexity** (polynomial degree $M$) affects:
- Training error
- Test error (generalization)

This demonstrates the classic **bias-variance tradeoff** and **overfitting** phenomenon.

## Data Generation

The underlying true function is:
$$f(x) = \sin(2\pi x)$$

We generate noisy observations:
$$t = \sin(2\pi x) + \epsilon \quad \text{where} \quad \epsilon \sim \mathcal{N}(0, \sigma^2)$$

Parameters:
- $\sigma = 0.3$ (noise standard deviation)
- $x \sim \mathcal{U}(0, 1)$ (uniform distribution)
- Training set: $N_{train} = 10$ samples
- Test set: $N_{test} = 100$ samples

## Polynomial Model Definition

We model the data using a polynomial of degree $M$:
$$y(x, \mathbf{w}) = w_0 + w_1 x + w_2 x^2 + \cdots + w_M x^M = \sum_{j=0}^{M} w_j x^j$$

where:
- $\mathbf{w} = [w_0, w_1, \ldots, w_M]^T$ is the weight vector
- $M$ is the polynomial degree (model complexity parameter)

Key insight: Although the model is polynomial in $x$, it is **linear in the parameters** $\mathbf{w}$, making this a linear regression problem.

## Design Matrix Construction

For $N$ data points $\{x_1, x_2, \ldots, x_N\}$, we construct the **design matrix** $\mathbf{\Phi}$:

$$\mathbf{\Phi} = \begin{bmatrix} 1 & x_1 & x_1^2 & \cdots & x_1^M \\ 1 & x_2 & x_2^2 & \cdots & x_2^M \\ \vdots & \vdots & \vdots & \ddots & \vdots \\ 1 & x_N & x_N^2 & \cdots & x_N^M \end{bmatrix}$$

where:
- $\mathbf{\Phi}$ is an $N \times (M+1)$ matrix
- Each row $i$: $\phi(x_i)^T = [1, x_i, x_i^2, \ldots, x_i^M]$
- The $j$-th column contains the $j$-th power of all input values

The predictions can then be written as:
$$\mathbf{y} = \mathbf{\Phi} \mathbf{w}$$

## Least Squares Solution

### Cost Function
We minimize the sum-of-squares error:
$$J(\mathbf{w}) = \frac{1}{2} \sum_{i=1}^{N} (y(x_i, \mathbf{w}) - t_i)^2 = \frac{1}{2} ||\mathbf{\Phi}\mathbf{w} - \mathbf{t}||^2$$

### Deriving the Optimal Solution

1. Expand the cost function:
$$J(\mathbf{w}) = \frac{1}{2} (\mathbf{\Phi}\mathbf{w} - \mathbf{t})^T(\mathbf{\Phi}\mathbf{w} - \mathbf{t})$$
$$= \frac{1}{2} (\mathbf{w}^T\mathbf{\Phi}^T\mathbf{\Phi}\mathbf{w} - 2\mathbf{w}^T\mathbf{\Phi}^T\mathbf{t} + \mathbf{t}^T\mathbf{t})$$

2. Take the gradient with respect to $\mathbf{w}$:
$$\nabla_{\mathbf{w}} J(\mathbf{w}) = \mathbf{\Phi}^T\mathbf{\Phi}\mathbf{w} - \mathbf{\Phi}^T\mathbf{t}$$

3. Set gradient to zero and solve:
$$\mathbf{\Phi}^T\mathbf{\Phi}\mathbf{w} = \mathbf{\Phi}^T\mathbf{t}$$

### Closed-Form Solution (Normal Equations)
$$\mathbf{w}^* = (\mathbf{\Phi}^T\mathbf{\Phi})^{-1}\mathbf{\Phi}^T\mathbf{t}$$

*Note: This solution exists and is unique when $\mathbf{\Phi}^T\mathbf{\Phi}$ is invertible. For high-degree polynomials with few data points, numerical issues may arise.*

## Error Metric: Root Mean Square Error

To evaluate model performance, we use the Root Mean Square (RMS) error:

$$E_{RMS} = \sqrt{\frac{2 \cdot J(\mathbf{w})}{N}} = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (y(x_i, \mathbf{w}) - t_i)^2}$$

where:
- $J(\mathbf{w}) = \frac{1}{2} \sum_{i=1}^{N} (y_i - t_i)^2$ is our cost function
- Dividing by $N$ allows comparison across datasets of different sizes
- Taking the square root gives an error in the same units as the target

We compute $E_{RMS}$ separately for:
- **Training set**: Measures how well the model fits the training data
- **Test set**: Measures how well the model generalizes to unseen data

## Training vs Test Evaluation

For each polynomial degree $M \in \{0, 1, 2, \ldots, 9\}$:

1. **Build** the design matrix $\mathbf{\Phi}_{train}$ using training inputs
2. **Solve** for optimal weights: $\mathbf{w}^* = (\mathbf{\Phi}_{train}^T\mathbf{\Phi}_{train})^{-1}\mathbf{\Phi}_{train}^T\mathbf{t}_{train}$
3. **Predict** on training set: $\mathbf{y}_{train} = \mathbf{\Phi}_{train}\mathbf{w}^*$
4. **Predict** on test set: $\mathbf{y}_{test} = \mathbf{\Phi}_{test}\mathbf{w}^*$
5. **Compute** $E_{RMS}^{train}$ and $E_{RMS}^{test}$

### Procedure Summary

| Step | Formula | Description |
|------|---------|-------------|
| Design Matrix | $\Phi_{ij} = x_i^j$ | Powers of input features |
| Solve Weights | $\mathbf{w}^* = (\Phi^T\Phi)^{-1}\Phi^T\mathbf{t}$ | Closed-form solution |
| Predict | $\mathbf{y} = \Phi\mathbf{w}^*$ | Matrix-vector product |
| Error | $E_{RMS} = \sqrt{\frac{1}{N}||\mathbf{y} - \mathbf{t}||^2}$ | RMS error |

## Expected Results: Overfitting Behavior

### Typical Error Curves

As polynomial degree $M$ increases:

| $M$ | Training Error | Test Error | Interpretation |
|-----|----------------|------------|----------------|
| Low (0-1) | High | High | **Underfitting**: Model too simple |
| Medium (2-3) | Medium | Low | **Good fit**: Captures pattern |
| High (7-9) | Very Low | Very High | **Overfitting**: Fits noise |

### Key Observations

1. **Training error** monotonically decreases as $M$ increases
   - More parameters = more flexibility to fit training data
   - At $M = N - 1$, polynomial passes through all training points (zero training error)

2. **Test error** follows a U-shaped curve:
   - Initially decreases (better model fits underlying pattern)
   - Eventually increases (model starts fitting noise)

3. **The gap** between training and test error indicates overfitting
   - Small gap: Good generalization
   - Large gap: Overfitting

### Why Overfitting Occurs

With limited training data ($N = 10$) and high model complexity ($M = 9$):
- The model has 10 parameters to fit 10 data points
- It can perfectly interpolate the noisy training data
- But the learned function oscillates wildly between data points
- Poor generalization to new test points

## Effect of Training Set Size

When we increase training data to $N = 100$:

1. **Overfitting is reduced** - more data constraints the model
2. **Test error stays lower** for higher-degree polynomials
3. **Gap closes** - training and test error become more similar

This demonstrates that **more data is a natural regularizer** against overfitting.

### Rule of Thumb

To avoid overfitting: $N >> M + 1$ (number of samples much greater than number of parameters)